In [84]:
print('hello')

hello


In [85]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import numpy as np
import time
import intel_extension_for_pytorch as ipex
print(torch.__version__)
device = 'xpu' if torch.xpu.is_available() else 'cpu'
print(f'Using device: {device}')

2.8.0+xpu
Using device: xpu


In [86]:
torchrand1 = torch.randn(100, 100, 100, 100, device=device)
torchrand2 = torch.randn(100, 100, 100, 100, device=device)
np_rand1 = torch.randn(100, 100, 100, 100)
np_rand2 = torch.randn(100, 100, 100, 100)

start = time.time()
rand = torch.matmul(torchrand1, torchrand2)
end = time.time()
print(f'gpu torch time: {end - start} seconds')

start = time.time()
rand = np.multiply(np_rand1, np_rand2)
end = time.time()
print(f'cpu numpy time: {end - start} seconds')

gpu torch time: 0.011606454849243164 seconds
cpu numpy time: 0.06210207939147949 seconds


C:\Users\Blight\AppData\Local\Temp\ipykernel_3196\4232043540.py:12: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  rand = np.multiply(np_rand1, np_rand2)


In [87]:
block=8
batch=4
max_iters=10000
# eval_interval=2500
eval_iters=250
learning_rate=3e-4

In [88]:
with open('frankenstein.txt', 'r', encoding='utf-8') as f:
    text=f.read()
    chars = sorted(list(set(text)))
print(text[:200])
print(f'Total characters: {chars}')
vocab_size = len(chars)

Letter 1

_To Mrs. Saville, England._


St. Petersburgh, Dec. 11th, 17—.


You will rejoice to hear that no disaster has accompanied the
commencement of an enterprise which you have regarded with suc
Total characters: ['\n', ' ', '!', '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'æ', 'è', 'é', 'ê', 'ô', '—', '‘', '’', '“', '”', '\ufeff']


In [89]:
stringtoint={ch:i for i,ch in enumerate(chars)}
inttostring={i:ch for i,ch in enumerate(chars)}
encode=lambda s: [stringtoint[c] for c in s]
decode=lambda l: ''.join([inttostring[i] for i in l])

print(encode('hello world'))
print(decode(encode('hello world')))

[54, 51, 58, 58, 61, 1, 69, 61, 64, 58, 50]
hello world


In [90]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([418920]) torch.int64
tensor([83, 32, 51, 66, 66, 51, 64,  1,  9,  0,  0, 46, 39, 61,  1, 33, 64, 65,
         7,  1, 38, 47, 68, 55, 58, 58, 51,  5,  1, 25, 60, 53, 58, 47, 60, 50,
         7, 46,  0,  0,  0, 38, 66,  7,  1, 36, 51, 66, 51, 64, 65, 48, 67, 64,
        53, 54,  5,  1, 24, 51, 49,  7,  1,  9,  9, 66, 54,  5,  1,  9, 15, 78,
         7,  0,  0,  0, 43, 61, 67,  1, 69, 55, 58, 58,  1, 64, 51, 56, 61, 55,
        49, 51,  1, 66, 61,  1, 54, 51, 47, 64])


In [91]:
n=int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]
print(train_data.shape, val_data.shape)

def get_batch(split):
    data = train_data if split =='train' else val_data
    ix = torch.randint(len(data) - block,(batch,))
    # print(ix)
    x = torch.stack([data[i:i+block] for i in ix])
    y = torch.stack([data[i+1:i+block+1] for i in ix])
    x=x.to(device); y=y.to(device)
    return x,y 

x,y = get_batch('train')
print('inputs:')
print(x)
print('targets:')
print(y)   

torch.Size([335136]) torch.Size([83784])
inputs:
tensor([[60,  1, 62, 47, 67, 65, 51, 50],
        [60,  1, 51, 60, 50,  1, 66, 61],
        [55, 65, 49, 55, 62, 58, 51,  1],
        [67, 60, 50,  1, 29,  1, 65, 47]], device='xpu:0')
targets:
tensor([[ 1, 62, 47, 67, 65, 51, 50,  5],
        [ 1, 51, 60, 50,  1, 66, 61,  1],
        [65, 49, 55, 62, 58, 51,  1, 61],
        [60, 50,  1, 29,  1, 65, 47, 69]], device='xpu:0')


In [92]:
x = train_data[:block]
y = train_data[1:block+1]

for i in range(block):
    context = x[:i+1]
    target = y[i]
    print(f'when input is {context}, target is {target}')

when input is tensor([83]), target is 32
when input is tensor([83, 32]), target is 51
when input is tensor([83, 32, 51]), target is 66
when input is tensor([83, 32, 51, 66]), target is 66
when input is tensor([83, 32, 51, 66, 66]), target is 51
when input is tensor([83, 32, 51, 66, 66, 51]), target is 64
when input is tensor([83, 32, 51, 66, 66, 51, 64]), target is 1
when input is tensor([83, 32, 51, 66, 66, 51, 64,  1]), target is 9


In [93]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx)
        
        if targets is None:
            loss=None
        else:
            B,T,C=logits.shape
            logits=logits.view(B*T,C)
            targets=targets.view(B*T)
            loss=F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for i in range(max_new_tokens):
            logits, loss = self.forward(idx)
            logits = logits[:, -1, :]
            probs =F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=-1)
        return idx
    
model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars=decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


h”BDJsOWP 821—Cx2aD:V!MED-oS’’jBJrP7oxY]0êCpdk1)Ep.sèMk1æEPz(Rj﻿4,EAuz Aê!AL.ezRdYhO(q h4b8lBc3u“?5,UC!;Fs0)﻿Y“
”hæD“dq2.k1bA—vxY﻿﻿vnPqTtNA1fUw[a﻿6C-_!qoicp2aéGdêeUyL)5gêô—:tovqhh”Mt]æTAgê
KPæ83ijNsIrtèRKigôkLsh7t8o]Tè[v9xi0)ScmzRG_A—AkMq58R!;jsh5Ièèe”A(0V4, aBGJwé9vl9﻿hln3ér﻿Syæa4bW.J—UYôml6opW :HbérI
G8lt(fè,_.’ôMEC!ôTê—ru!z7zR,]6D1-)6Oi00pq”rgêd“KS0éæ,!kr:HU([LHV;.u:uzRtx 6LJ7L5PV5K__Meqbd3D0c tèdi0êe7u﻿﻿]1(R﻿ôtog aWn)è1K532i0))O
-‘ubTtrv6KJfa7êeKi1( YsiL8ccé;1YP—]‘386Va[èkM5oI._]6INè﻿v9NBhn—


In [94]:
@torch.no_grad()
def estimate_loss():
    out={}
    model.eval()
    for split in ['train','val']:
        losses=torch.zeros(eval_iters)
        for i in range(eval_iters):
            X,Y=get_batch(split)
            logits, loss = model(X, Y)
            losses[i]=loss.item()
        out[split]=losses.mean()
    model.train()
    return out

In [95]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iteration in range(max_iters):
    if iteration%eval_iters==0:
        losses=estimate_loss()
        print(f"step {iteration}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
    
    xb,yb=get_batch('train')
    
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
# print(loss.item())

step 0: train loss 4.9421, val loss 4.9620
step 250: train loss 4.8894, val loss 4.8831
step 500: train loss 4.8007, val loss 4.8127
step 750: train loss 4.7303, val loss 4.7498
step 1000: train loss 4.6785, val loss 4.6976
step 1250: train loss 4.6063, val loss 4.6128
step 1500: train loss 4.5421, val loss 4.5589
step 1750: train loss 4.4946, val loss 4.5022
step 2000: train loss 4.4148, val loss 4.4424
step 2250: train loss 4.3696, val loss 4.3964
step 2500: train loss 4.3396, val loss 4.3177
step 2750: train loss 4.2795, val loss 4.2842
step 3000: train loss 4.1952, val loss 4.2408
step 3250: train loss 4.1522, val loss 4.1631
step 3500: train loss 4.0843, val loss 4.1223
step 3750: train loss 4.0536, val loss 4.0737
step 4000: train loss 3.9927, val loss 4.0146
step 4250: train loss 3.9504, val loss 3.9731
step 4500: train loss 3.9186, val loss 3.9135
step 4750: train loss 3.8626, val loss 3.8631
step 5000: train loss 3.8054, val loss 3.8392
step 5250: train loss 3.7662, val loss 3

In [96]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars=decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


JWOY3tmyæG_wBô ccay, to, w37RG_wd
aVô—a_a
gew(q h,V(2i)kPqYj0Cd4UVicof ôôjE44o sa a[aPhld,wa.ôjug’
t—ôjOb8éPV4GxYMf9zRI6ôyhi
V(Ts—B?GR)cenofêôj8:lwUr_tha7)PwBêhæ.’ô2Lx wdgDuJqTtmhGR1v5JH0Wrd9O3Lbutudê1êôP9FK(Y2054, strh”rPVégêhl;:fonKF?w?erdEx;xWJ6WK7L_﻿8(0O4urwB39﻿Oq-]B0m9eIomye cqYj[﻿sizècoctrn RtuTaD[kéLWJqy spé;RvuR7bl;P_ as?0E-pra7MmM“:)[alG,ing”T5L, the7Læ‘?VKW(BJinefaVD﻿LVD w1,TFhll;18GGêô
qherav__44t_hu Vz(fdoue7 hendlæ BD
voR1llif sabf20Po awhld0Aa s.:VUVKNT)TanYin aco T)gh we.EôYht
﻿êT
